# Jane Street Puzzle - April 2025: Sum One, Somewhere

This notebook provides a step-by-step solution to the Jane Street puzzle from April 2025.

## Problem Statement

For a fixed value of **p**, each node of an infinite complete binary tree is independently labeled **0** with probability **p**, and **1** otherwise. 

What is the value of **p** for which there is exactly a **1/2 probability** that there exists an infinite path down the tree with a sum of labels at most 1? 

The final answer should be accurate to 10 decimal places.

## Step 1: Setting up the Recursive Equations

Let's define two key probabilities to model this problem recursively:

-   **f(p)**: The probability that there exists at least one infinite path starting from the root with a sum of labels of at most 1.
-   **g(p)**: The probability that there exists at least one infinite path starting from the root with a sum of labels equal to 0 (an all-0s path).

We can express **f(p)** by considering the label of the root node:

1.  The root is labeled **0** (with probability **p**): For a valid path to exist from the root, at least one of its two subtrees must contain a path with a sum of at most 1. The probability that a single subtree *does not* contain such a path is `(1 - f(p))`. Therefore, the probability that *neither* subtree contains such a path is `(1 - f(p))²`.

2.  The root is labeled **1** (with probability **1-p**): If the root is 1, we have already used our budget of a single '1'. Therefore, any valid path must continue through a subtree with an all-0s path. The probability of finding an all-0s path in a subtree is `g(p)`.

Combining these cases gives us the equation for **f(p)**:
$$ f(p) = p \cdot [1 - (1 - f(p))^2] + (1 - p) \cdot [1 - (1 - g(p))^2] $$

## Step 2: Solving for g(p)

Now, let's find an expression for **g(p)**, the probability of an all-0s path.

This gives the recursive relation:
$$ g(p) = p \cdot [1 - (1 - g(p))^2] $$

This is a classic equation from percolation theory.
$$ g = p(2g - g^2) $$
$$ pg^2 - 2pg + g = 0 $$
$$ g(pg - 2p + 1) = 0 $$

This equation has two solutions: `g = 0` or `g = (2p - 1) / p`.

The non-zero solution `g(p) = (2p - 1) / p` is physically meaningful only when `p > 1/2`.

First, let's test the assumption that `p <= 1/2`.
In this case, `g(p) = 0`, and our equation for `f(p)` simplifies to:
$$ f(p) = p[1 - (1 - f(p))^2] $$
The problem states that `f(p) = 1/2`.
Substituting this in:
$$ \frac{1}{2} = p[1 - (1 - \frac{1}{2})^2] = p[1 - \frac{1}{4}] = \frac{3}{4}p $$
$$ p = \frac{1}{2} \cdot \frac{4}{3} = \frac{2}{3} $$
This result, `p = 2/3`, contradicts our assumption that `p <= 1/2`.
Therefore, we must operate in the regime where `p > 1/2` and `g(p) = (2p - 1) / p`.

## Step 3: Deriving the Final Cubic Equation

Now we substitute `g(p) = (2p - 1) / p` and `f(p) = 1/2` into our main equation.

$$ f(p) = p [1 - (1 - f(p))^2] + (1 - p) [1 - (1 - g(p))^2] $$

Let's evaluate each term:
- The first term is `p[1 - (1 - 1/2)²] = p(3/4)`.
- For the second term, `1 - g(p) = 1 - (2p - 1)/p = (p - 2p + 1)/p = (1 - p)/p`.
- So, `1 - (1 - g(p))² = 1 - ((1-p)/p)² = 1 - (1 - 2p + p²)/p² = (p² - 1 + 2p - p²)/p² = (2p - 1)/p²`.
- The second term is `(1 - p) * (2p - 1)/p²`.

Substituting everything back with `f(p) = 1/2`:
$$ \frac{1}{2} = \frac{3}{4}p + \frac{(1 - p)(2p - 1)}{p^2} $$

To solve for `p`, we can multiply by `4p²` to clear the denominators:
$$ 2p^2 = 3p^3 + 4(1 - p)(2p - 1) $$
$$ 2p^2 = 3p^3 + 4(2p - 1 - 2p^2 + p) $$
$$ 2p^2 = 3p^3 + 4(3p - 2p^2 - 1) $$
$$ 2p^2 = 3p^3 + 12p - 8p^2 - 4 $$

Rearranging the terms gives us the final cubic equation:
$$ 3p^3 - 10p^2 + 12p - 4 = 0 $$

## Step 4: Numerical Solution with Python

The cubic equation `3p³ - 10p² + 12p - 4 = 0` does not have simple rational roots.
We can solve it numerically using the Newton-Raphson method, which is an iterative approach to find the roots of a real-valued function.

The formula for the Newton-Raphson method is:
$$ p_{n+1} = p_n - \frac{h(p_n)}{h'(p_n)} $$

Where:
- `h(p) = 3p³ - 10p² + 12p - 4`
- `h'(p) = 9p² - 20p + 12`

We need an initial guess. From our analysis, we know the root lies between `0.5` and `2/3`.
A good starting point is `0.53`.

In [ ]:
def h(p):
    """The cubic equation we need to solve."""
    return 3 * p**3 - 10 * p**2 + 12 * p - 4

def h_prime(p):
    """The derivative of the cubic equation."""
    return 9 * p**2 - 20 * p + 12

def solve_newton_raphson(initial_guess, tolerance=1e-12, max_iterations=100):
    """Solves for the root using the Newton-Raphson method."""
    p = initial_guess
    
    for i in range(max_iterations):
        h_p = h(p)
        h_prime_p = h_prime(p)
        
        if abs(h_prime_p) < 1e-15:
            raise ValueError("Derivative is zero, cannot continue.")
            
        p_new = p - h_p / h_prime_p
        
        if abs(p_new - p) < tolerance:
            print(f"Converged after {i+1} iterations.")
            return p_new
        
        p = p_new
        
    raise ValueError("Solution did not converge within max iterations.")

# Our analysis showed the root is near 0.53
initial_guess = 0.53

solution = solve_newton_raphson(initial_guess)

print(f"\nThe value of p accurate to 10 decimal places is: {solution:.10f}")

## Conclusion
The value of **p** for which the probability of finding an infinite path with at most one '1' is exactly 1/2 is the root of the cubic equation `3p³ - 10p² + 12p - 4 = 0` that lies between `1/2` and `1`. The numerical solution gives:
$$ p \approx  0.5306035754 $$